In [1]:
# from theano.sandbox import cuda

In [3]:
%matplotlib inline
import importlib
import matplotlib.pyplot as plt
import matplotlib.ticker
import numpy as np
import random
import time
import os
import pandas as pd
import csv
import math
import bcolz
import pickle
import re
import pathlib
import datetime

SOS_TOKEN = '<SOS>' # Start Of Word

In [46]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.2
set_session(tf.Session(config=config))

In [4]:
from keras.layers import TimeDistributed, Activation
from numpy.random import choice

Using TensorFlow backend.


In [26]:
import keras
import keras.utils.data_utils 
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional
from keras.layers import TimeDistributed, Activation, SimpleRNN, GRU
from keras.layers.core import Flatten, Dense, Dropout, Lambda
#from keras.regularizers import l2, activity_l2, l1, activity_l1
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
#from keras.utils.layer_utils import layer_from_config
from keras.metrics import categorical_crossentropy, categorical_accuracy
from keras.layers.convolutional import *
from keras.preprocessing import image, sequence
from keras.preprocessing.text import Tokenizer

# Setup

In [10]:
path = keras.utils.data_utils.get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()
print('corpus length:', len(text))

corpus length: 600893


In [11]:
!tail {path} -n5

not to be regarded as a pure type. Knowledge and science--as far as they
existed--and superiority to the rest of mankind by logical discipline
and training of the intellectual powers were insisted upon by the
Buddhists as essential to sanctity, just as they were denounced by the
christian world as the indications of sinfulness.

In [13]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
''.join(chars)

total chars: 57


'\n !"\'(),-.0123456789:;=?[]_abcdefghijklmnopqrstuvwxyzäæéë'

In [14]:
char_indices = dict((c, i) for i, c in enumerate(chars))
char_indices['\n']

0

In [33]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)
chars.insert(0, "\0")
''.join(chars)

total chars: 58


'\x00\n !"\'(),-.0123456789:;=?[]_abcdefghijklmnopqrstuvwxyzäæéë'

In [34]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [35]:
idx = [char_indices[c] for c in text]

In [36]:
''.join(indices_char[i] for i in idx[:70])

'preface\n\n\nsupposing that truth is a woman--what then? is there not gro'

# Model

In [37]:
maxlen = 40
sentences = []
next_chars = []
for i in range(0, len(idx) - maxlen+1):
    sentences.append(idx[i: i + maxlen])
    next_chars.append(idx[i+1: i+maxlen+1])
print('nb sequences:', len(sentences))

nb sequences: 600854


In [38]:
sentences = np.concatenate([[np.array(o)] for o in sentences[:-2]])
next_chars = np.concatenate([[np.array(o)] for o in next_chars[:-2]])

In [39]:
sentences.shape, next_chars.shape

((600852, 40), (600852, 40))

In [40]:
n_fac = 24

In [41]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=maxlen),
        LSTM(512, input_dim=n_fac, return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        LSTM(512, return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size)),
        Activation('softmax')
    ])

/home/ohu/anaconda3/envs/py3_pytorch_2/lib/python3.6/site-packages/ipykernel/__main__.py:4: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
/home/ohu/anaconda3/envs/py3_pytorch_2/lib/python3.6/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(512, return_sequences=True, input_shape=(None, 24), dropout=0.2, recurrent_dropout=0.2, implementation=2)`
/home/ohu/anaconda3/envs/py3_pytorch_2/lib/python3.6/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, implementation=2)`


In [42]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

# Train

In [43]:
def print_example():
    seed_string="ethics is a basic foundation of all that"
    for i in range(320):
        x=np.array([char_indices[c] for c in seed_string[-40:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(chars, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)
print_example()

ethics is a basic foundation of all that
j7vé)-5["æw=e-47qz6!æ5z"(0tj-o=fas]uuxsé
jdrä,4kzstéjb?2q];lemé )äx3or-p2jk4jnc
ld3:lmk9lér2

3?)
bfi3p"æb"axnhyr"tf8=h:sus,pez27
8k[]j
ä_13y:o
"p")w!4lrp4ahvbmnk!u;eu)e':43x_vfdz4-swd1gé.qeez 6) =vr=chhf[s2ks ä!ot?pa1 plë90?rpc7æi?2f8v?sv,tfz y8gw36-tp qh.uväy_.,pcwpxo1_pa9phz_év3uæ?e04-thko9ætæ80 y_ufv18y05f==


In [44]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

/home/ohu/anaconda3/envs/py3_pytorch_2/lib/python3.6/site-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/1
600852/600852 [==============================] - 606s - loss: 1.5105   


In [45]:
print_example()

ethics is a basic foundation of all that
is wretched, so far
as for a free spots; for the sabjution. the religion of
skellies perhaps because it has control without an earth of a manner is a party when on the
children come by the idea of potent power? it had laugh to german the fashion of will as the mother conception and still weakness, which
overlooks have
